In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [3]:
from dataset import MyDataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", model_max_length=100)
train_dataset = MyDataset("./data/laptop.train.txt", tokenizer)
eval_dataset = MyDataset("./dataset/rest.validation.txt", tokenizer)
test_dataset = MyDataset("./dataset/rest.test.txt", tokenizer)

In [4]:
from typing import List, Optional, Tuple, Union

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import TokenClassifierOutput

TAGS = ['O', 'B-POS', 'B-NEG', 'B-NEU', 'I-POS', 'I-NEG', 'I-NEU']
def id2label(predict: List[List[int]], gold: List[List[int]]):
    gold_Y: List[List[str]] = []
    pred_Y: List[List[str]] = []
    for _pred, _gold in zip(predict, gold):
        assert len(_gold) == len(_pred)
        gold_list = [TAGS[_gold[i]] for i in range(len(_gold)) if _gold[i] != -1]
        pred_list = [TAGS[_pred[i]] for i in range(len(_gold)) if _gold[i] != -1]
        gold_Y.append(gold_list)
        pred_Y.append(pred_list)
    return pred_Y, gold_Y

In [6]:
import numpy as np
import torch
from transformers import (EvalPrediction, IntervalStrategy, Trainer,
                          TrainingArguments, set_seed)

from eval import absa_evaluate
from model import BertForTokenClassification
from optimization import BertAdam, WarmupLinearSchedule

training_args = TrainingArguments(output_dir="test_trainer")
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=7)

def compute_metrics(eval_pred: EvalPrediction):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    predictions = np.argmax(logits, axis=-1)
    pred_Y, gold_Y = id2label(predictions, labels)
    p, r, f1 = absa_evaluate(pred_Y, gold_Y)
    return {"precision": p, "recall": r, "micro-f1": f1}
param_optimizer = [(k, v) for k, v in model.named_parameters() if v.requires_grad == True]
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy=IntervalStrategy.EPOCH, report_to='all', num_train_epochs=3)
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
        'weight_decay': 0.01
    },
    {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
        'weight_decay': 0.0
    }
]

class CustomTrainer(Trainer):
    def create_optimizer_and_scheduler(self, num_training_steps: int):
        self.optimizer = BertAdam(optimizer_grouped_parameters,
                            lr=3e-5,
                            warmup=0.1,
                            t_total=num_training_steps)
        self.create_scheduler(num_training_steps=num_training_steps, optimizer=self.optimizer)
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.predict(test_dataset)

***** Running training *****
  Num examples = 3045
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 573
/home/oyf/anaconda3/envs/bert-uda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,Micro-f1
1,No log,0.460296,0.666667,0.190703,0.296567
2,No log,0.525376,0.679654,0.187128,0.293455
3,0.145600,0.519360,0.640741,0.206198,0.311989


***** Running Evaluation *****
  Num examples = 776
  Batch size = 16
***** Running Evaluation *****
  Num examples = 776
  Batch size = 16
Saving model checkpoint to test_trainer/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
/home/oyf/anaconda3/envs/bert-uda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 776
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 2158
  Batch size = 16


PredictionOutput(predictions=array([[[ 7.20796585e+00,  6.59066558e-01, -1.83504248e+00, ...,
         -6.97364092e-01, -2.06241584e+00, -1.81653881e+00],
        [ 7.48648262e+00,  5.38699746e-01, -1.99433732e+00, ...,
         -7.19753623e-01, -2.10051537e+00, -1.78098714e+00],
        [ 5.40427065e+00,  2.76849008e+00, -1.15224111e+00, ...,
         -3.86806130e-01, -2.56797552e+00, -2.19729066e+00],
        ...,
        [ 6.79556179e+00,  8.88111115e-01, -1.95088303e+00, ...,
         -6.63545251e-01, -2.01610208e+00, -1.69472635e+00],
        [ 7.08384418e+00,  6.76379383e-01, -1.98597538e+00, ...,
         -7.13197708e-01, -2.00047469e+00, -1.70323718e+00],
        [ 6.83704758e+00,  8.26546550e-01, -1.97394776e+00, ...,
         -6.80289209e-01, -2.00996804e+00, -1.67529786e+00]],

       [[ 7.39760780e+00, -4.21143621e-01, -1.81070447e+00, ...,
         -8.17592800e-01, -1.46576190e+00, -1.38333344e+00],
        [ 8.15382481e+00, -9.67814684e-01, -2.03173470e+00, ...,
         